## Compute the distances among cell types in Niche4.

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import scanpy as sc
import scimap as sm
import anndata as ad
import stlearn as st
import matplotlib.pyplot as plt

##### Set working directory for analysis

In [ ]:
cwd = '/media/bio/Disk/Research Data/EBV/omicverse'
os.chdir(cwd)
updated_dir = os.getcwd()
print("Updated working directory: ", updated_dir)

from pathlib import Path
saving_dir = Path('Results/10.NPC_ST_Analysis')
saving_dir.mkdir(parents=True, exist_ok=True)

## Loading the data

#### Reading in annotated AnnData object

In [ ]:
adata = sc.read_h5ad("Processed Data/GSE206245_NPC_ST_Cluster_Tangram.h5ad")
adata

In [ ]:
import numpy as np
import pandas as pd

celltype_cols = [
    "B","C1QC+ Macro","Monocyte","Fibroblast","IL1B+ Macro","IgM+ plasma-like",
    "Mast","NK","Neutrophil","Epithelial","Plasma","SPP1+ Macro","T","Tumor","cDC","pDC"
]

df = adata.obs[celltype_cols].astype(float).fillna(0.0)

max_prop = df.max(axis=1)
wta = df.idxmax(axis=1)

min_prop = 0.0  
assigned = wta.where(max_prop >= min_prop, other="Unassigned")

priority = ["Tumor", "Epithelial", "SPP1+ Macro", "C1QC+ Macro", "IL1B+ Macro",
            "Monocyte", "T", "B", "NK", "Neutrophil", "Plasma", "IgM+ plasma-like",
            "Fibroblast", "Mast", "cDC", "pDC"]

def break_tie(row):
    vals = row.values
    m = vals.max()
    winners = [row.index[i] for i, v in enumerate(vals) if v == m]
    for p in priority:
        if p in winners:
            return p
    return winners[0]

# assigned = df.apply(break_tie, axis=1).where(max_prop >= min_prop, other="Unassigned")

adata.obs["CellType_WTA"] = pd.Categorical(assigned)
adata.obs["CellType_WTA_prop"] = max_prop.astype(float)

In [ ]:
lib_ids = ['NPC_ST09', 'NPC_ST10', 'NPC_ST16', 'NPC_ST17', 'NPC_ST18', 'NPC_ST19']
mask = (adata.obs['scNiche'] == 'Niche4') & (adata.obs['library_id'].isin(lib_ids))
adata_Niche4 = adata[mask].copy()

In [ ]:
if 'spatial' in adata_Niche4.obsm:
    adata_Niche4.obs['X'] = adata_Niche4.obsm['spatial'][:, 0]
    adata_Niche4.obs['Y'] = adata_Niche4.obsm['spatial'][:, 1]

adata_Niche4.obs['CellType_WTA'] = adata_Niche4.obs['CellType_WTA'].astype('category')

### Compute the distances among cell types.

In [ ]:
adata_Niche4 = sm.tl.spatial_distance(
    adata_Niche4,
    x_coordinate='X',
    y_coordinate='Y',
    phenotype='CellType_WTA',
    imageid='library_id'
)

In [ ]:
adata_Niche4

We'll utilize built-in plotting functions for visualization; however, for those interested in conducting additional analysis or custom plotting of these distances, the results can be found in adata.uns['spatial_distance'].

In [ ]:
# This is one of the most complicated plotting functions in the package
# as I packed a lot of options into one (see the documentation)
# I will try and split this into multiple functions in the future
# To start- let's get an overview with a heatmap
sm.pl.spatial_distance(adata_Niche4, method='heatmap', phenotype='CellType_WTA', imageid='library_id', heatmap_summarize=True, figsize=(8,7))
sm.pl.spatial_distance(adata_Niche4, method='heatmap', phenotype='CellType_WTA', imageid='library_id', heatmap_summarize=True, figsize=(8,7),
                       saveDir='Results/10.NPC_ST_Analysis', fileName= 'SpatialDistance_Heatmap_Niche4.pdf',)

In [ ]:
# Numeric plot showing distance from one phenotype to all others
sm.pl.spatial_distance(adata_Niche4, method='numeric', distance_from='Tumor', phenotype='CellType_WTA', imageid='scNiche', log=True, height=5.5, aspect=12/8)

sm.pl.spatial_distance(adata_Niche4, method='numeric', distance_from='Tumor', phenotype='CellType_WTA', imageid='scNiche', log=True, height=5.25, aspect=12/8,
                       saveDir='Results/10.NPC_ST_Analysis', fileName= 'SpatialDistance_Numeric_Tumor_Niche4.pdf')



**<span style="font-size:16px;">Session information：</span>**

In [ ]:
import sys
import platform
import pkg_resources

# Get Python version information
python_version = sys.version
# Get operating system information
os_info = platform.platform()
# Get system architecture information
architecture = platform.architecture()[0]
# Get CPU information
cpu_info = platform.processor()
# Print Session information
print("Python version:", python_version)
print("Operating system:", os_info)
print("System architecture:", architecture)
print("CPU info:", cpu_info)

# Print imported packages and their versions
print("\nImported packages and their versions:")
for package in pkg_resources.working_set:
    print(package.key, package.version)